In [1]:
import codecs
from bs4 import BeautifulSoup
import os
import re
import pandas as pd

In [2]:
def remove_multiple_space(string:str):
    
    return re.sub('\n+', '\n', string)

In [3]:
def pad_list(lst:[], size:int):
    new_lst = lst + [''] * (size - len(lst))
    return new_lst

In [4]:
def parse_attribute(att:str):
    att2 = att.strip()
    sp_2 = 'Attribute Definition:|Attribute Definition Source:|Attribute Domain Values:'
    att3 = re.split(sp_2, att2)
    att4 = [a.strip() for a in att3]
    att5 = pad_list(att4, 4)
    return att5
    

In [5]:
def meta_list(html_string):
    attribute_spl = "Attribute:\nAttribute Label:"
    atts = re.split(attribute_spl, html_string)
    result = [parse_attribute(a) for a in atts]
    return result

In [8]:
def html_to_df(year:int):
    file_name = ''
    if year == 2002:
        file_name = '2002.htm'
    else:
        file_name = '2003.html'
        
    m_path = '../raw_metadata'
    samp = os.path.join(m_path, file_name)
    
    with open(samp, 'rb') as fp:
        soup = BeautifulSoup(fp, 'html')
    
    txt = soup.get_text('\n')
    t2 = remove_multiple_space(txt)
    l = meta_list(t2)
    
    cols = ['field_name', 'definition', 'source', 'domain_definitions']
    df = pd.DataFrame(l, columns = cols)
    df['year'] = year
    return df

In [11]:
metadata_2003_2004=pd.concat(list(map(html_to_df, [2003,2004])))

In [15]:
# metadata_2003_2004
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '../secrets/secret.json'
bq_client = bigquery.Client()


table_id = "pluto-panel.raw.metadata_2002_2003"

job = bq_client.load_table_from_dataframe(
    metadata_2003_2004, table_id
)
job.result() 